Kaggle Competition: https://www.kaggle.com/competitions/asl-fingerspelling

## Exploring Datasets

In [6]:
import pandas as pd
import pyarrow.parquet as pq
import os

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#Looking at information within train.csv ans supplemental_metadata.csv
df_train = pd.read_csv('./dataset complete/train.csv')
df_supp_mat = pd.read_csv('./dataset complete/supplemental_metadata.csv')

In [ ]:
df_train.head()

In [ ]:
len(df_train)

In [ ]:
df_supp_mat.head()

In [ ]:
len(df_supp_mat)

From [here](https://www.kaggle.com/competitions/asl-fingerspelling/discussion/416507https://www.kaggle.com/competitions/asl-fingerspelling/discussion/416507) supplementary material is unnecessary. For now, do not use.

In [ ]:
# read in first parquet file
first_table = pq.read_table('./dataset complete/train_landmarks/105143404.parquet')
first_table = first_table.to_pandas()
first_table

In [ ]:
#We see for each sequence_id, the frames start again from 0
# man kann auch schlussfolgern in dieser datei sind 1000 sätze als vektoren dargestellt
zeros_table = first_table[first_table['frame'] == 0]
zeros_table

In [ ]:
#We see that the sequence_ids  are not in consecutive order the next sequence id after 1784552841 ist 1784574169
first_table[first_table.index == 1784552842]

Sequences are not sequentially numbered.

One sequence corresponds to one sentence.

Frame numberings restart at 0 for each sequence.

One sequence might contain n frames.

In [ ]:
#We print all existinmg columns
for column in first_table.columns.tolist():
    print(column)

<img src="https://camo.githubusercontent.com/7fbec98ddbc1dc4186852d1c29487efd7b1eb820c8b6ef34e113fcde40746be2/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f706f73655f747261636b696e675f66756c6c5f626f64795f6c616e646d61726b732e706e67" width="500">
<br>
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHiCz-z3dQ93Zh4p4zq-0dk4A535Dqp5OTHpnqoklaQQ&s" width="500">
<br>
<img src="https://raw.githubusercontent.com/google/mediapipe/a908d668c730da128dfa8d9f6bd25d519d006692/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png" width="500">

It might make sense to use only hands, mouth/lips, eyes, and eyebrows

In [ ]:
# Using isna() to select all rows with NaN under an entire DataFrame:
df_nan = first_table[first_table.isna().any(axis=1)]
df_nan

In [ ]:
# remove all pose columns except for the arm and wrist landmarks
to_remove = []
for column in first_table.columns.tolist():
    if 'pose' in column:
        if '11' in column:
            continue
        elif '12' in column:
            continue
        elif '13' in column:
            continue
        elif '14' in column:
            continue
        elif '15' in column:
            continue
        elif '16' in column:
            continue
        else:
            to_remove.append(column)

first_table = first_table.drop(to_remove, axis=1)
first_table

In [ ]:
df_nan = first_table[first_table.isna().any(axis=1)]
df_nan

In [ ]:
to_remove = []
for column in first_table.columns.tolist():
    if 'face' in column:
        to_remove.append(column)

first_table = first_table.drop(to_remove, axis=1)

df_nan = first_table[first_table.isna().any(axis=1)]
df_nan

In [ ]:
df_nan.index.unique()

In [ ]:
df_test = df_nan[df_nan.index == 1816603189]
df_test

In [ ]:
len(df_test)

In [ ]:
for column in first_table.columns.tolist():
    print(column)

We have to much nan values even when leaving out pose. Consequently, we need to find a good interpolation scheme.

In [ ]:

files = os.listdir('./dataset complete/train_landmarks')
len(files)

In [ ]:
# interpolation in cas it doesn't work how we want it to work
'''train_landmarks = remove_unwanted_columns(train_landmarks)
# merge the hand and arm data with the corresponding participant id and phrase
df = merge_datasets(train_landmarks, df_train)


phrases = df.sequence_id.unique()
removed_ids = []
df_concat = pd.DataFrame()
INTERPOLATION_NAN_GAP = 30
FILL_NAN_VALUE = -1
SKIP_VALUE = 10000000
pd.options.mode.chained_assignment = None
#for i in 0:
phrase_table = df[df['sequence_id'] == 1784574169]
#nan_count_left_hand = phrase_table['x_left_hand_0'].isna().sum()
#nan_count_right_hand = phrase_table['x_right_hand_0'].isna().sum()
#length_table = len(phrase_table)
#if nan_count_left_hand > 0.75*length_table and nan_count_right_hand > 0.75*length_table:
#    continue
for column in phrase_table.columns.tolist():
    #for column in ['x_left_hand_0', 'x_left_hand_1']:
    if(column == 'sequence_id') or (column == 'frame') or (column == 'participant_id') or (column == 'phrase'):
        continue
    nan_count = phrase_table[column].isnull().astype(int).groupby(phrase_table[column].notnull().cumsum()).cumsum()
    tmp = 0
    for x in range(phrase_table.index[0],phrase_table.index[0] + len(nan_count)-1):
        curr = nan_count[x]
        next = nan_count[x+1]
        if next > curr:
            tmp = next
            if (x == 541):
                if tmp > INTERPOLATION_NAN_GAP:
                    for j in range((x+2)-tmp, x+2):
                        nan_count[j] = SKIP_VALUE
                tmp = 0
        elif next == curr:
            tmp = 0
            continue
        elif (next < curr):
            if tmp > INTERPOLATION_NAN_GAP:
                for j in range(x-tmp, x+1):
                    nan_count[j] = SKIP_VALUE
            tmp = 0
    nan_count = nan_count.sort_index()
    phrase_table[column] = phrase_table[column].interpolate() \
        .where(nan_count < SKIP_VALUE, np.nan)



phrase_table = phrase_table.fillna(FILL_NAN_VALUE)
df_concat = pd.concat([df_concat, phrase_table], axis = 0)
df_concat'''

# Preprocessing TODOs
- goal: load first 30 parquet files
- for each parquet file select only relevant columns (remove face date and pose date except for arms)
- for each file remove NaNs (replace with -1 or interpolate if previous and following values are known)
- add participant number and label phrase to parquet dfs
- transform label phrase to array/string of numbers
- combine dfs
- export as one or two (hopefully not so) large pickle file

Tips:
- For each parquet file do preprocessing and save as pickle
- later combine multiple pickle files to one file
- to do preprocessing faster write preprocessing steps as function to only change name of file


Next build a Transformer.

# Preprocessing
## Train_Landmarks
Removing unnecessary columns

In [ ]:
def remove_unwanted_columns(df):
    # remove all pose columns except for the arm and wrist landmarks
    to_remove_pose = []
    for column in df.columns.tolist():
        if 'pose' in column:
            if '11' in column:
                continue
            elif '12' in column:
                continue
            elif '13' in column:
                continue
            elif '14' in column:
                continue
            elif '15' in column:
                continue
            elif '16' in column:
                continue
            else:
                to_remove_pose.append(column)

    df = df.drop(to_remove_pose, axis=1)


    to_remove_face = []
    for column in df.columns.tolist():
        if 'face' in column:
            to_remove_face.append(column)

    df = df.drop(to_remove_face, axis=1)

    return df

## Edit train_csv
change Strings to number arrays

In [ ]:
def exchange_string_with_numbers_array(df, characters):
    for i in range(0, len(df)):
        num_array = [characters.get(char, char) for char in df.iloc[i]['phrase']]
        df.at[i, 'phrase'] = num_array
    columns = ['sequence_id', 'participant_id', 'phrase']
    df_train_selected = df_train[columns]
    return df_train_selected

## Merge datasets
merge hand and arm data with participant id and phrase

In [ ]:
def merge_datasets(df, df_phrases):
    df_merged = pd.merge(df, df_phrases, on='sequence_id', how = 'left')
    return df_merged

## Interpolation of nan values
interpolate for nan gaps shorter than 30 following directly, else replace nan values with impossible number value (= out of frame)
This happens for each phrase dataset so we don't interpolate over multiple sentences

In [ ]:
#trivial interpolation function
INTERPOLATION_NAN_GAP = 30
FILL_NAN_VALUE = -1
SKIP_VALUE = 10000000
pd.options.mode.chained_assignment = None

def interpolate_dataset(df):
    phrases = df.sequence_id.unique()
    df_concat = pd.DataFrame()
    for i in phrases:
        phrase_table = df[df['sequence_id'] == i]
        nan_count_left_hand = phrase_table['x_left_hand_0'].isna().sum()
        nan_count_right_hand = phrase_table['x_right_hand_0'].isna().sum()
        length_table = len(phrase_table)
        if nan_count_left_hand > 0.75*length_table and nan_count_right_hand > 0.75*length_table:
            continue
        for column in phrase_table.columns.tolist():
            if(column == 'sequence_id') or (column == 'frame') or (column == 'participant_id') or (column == 'phrase'):
                continue
            nan_count = phrase_table[column].isnull().astype(int).groupby(phrase_table[column].notnull().cumsum()).cumsum()
            tmp = 0
            for x in range(phrase_table.index[0],phrase_table.index[0] + len(nan_count)-1):
                curr = nan_count[x]
                next = nan_count[x+1]
                if next > curr:
                    tmp = next
                    if (x == 541):
                        if tmp > INTERPOLATION_NAN_GAP:
                            for j in range(((x+2)-tmp), x+2):
                                nan_count[j] = SKIP_VALUE
                        tmp = 0
                elif next == curr:
                    tmp = 0
                    continue
                elif (next < curr):
                    if tmp > INTERPOLATION_NAN_GAP:
                        for j in range(x-tmp, x+1):
                            nan_count[j] = SKIP_VALUE
                    tmp = 0
            nan_count = nan_count.sort_index()
            phrase_table[column] = phrase_table[column].interpolate() \
                .where(nan_count < SKIP_VALUE, np.nan)

        df_concat = pd.concat([df_concat, phrase_table], axis = 0)

    df_concat = df_concat.fillna(FILL_NAN_VALUE)
    return df_concat

# Create preprocessed Files


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import os
import json
import numpy as np
import pickle

In [ ]:
df_train = pd.read_csv('./dataset complete/train.csv')

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)

df_train = exchange_string_with_numbers_array(df_train, characters)
df_train

In [ ]:
supp_train = pd.read_csv('./dataset complete/supplemental_metadata.csv')

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)

supp_train = exchange_string_with_numbers_array(supp_train, characters)
supp_train

In [ ]:
# read in first parquet file
train_landmarks = pq.read_table('./dataset complete/train_landmarks/105143404.parquet')
train_landmarks = train_landmarks.to_pandas()
train_landmarks

In [ ]:
# leave only columns with hand and arm data
train_landmarks = remove_unwanted_columns(train_landmarks)
# merge the hand and arm data with the corresponding participant id and phrase
df = merge_datasets(train_landmarks, df_train)
# for short nan gaps interpolate the values, else fill Nans with a placeholder number
df = interpolate_dataset(df)
# check if all nans have been removed
if len(df[df.isna().any(axis=1)]) != 0:
    print("Still NANs!")
# print df
df

Save as Pickle file:

In [ ]:
#file_path = './dataset complete/preprocessed_files/105143404.pkl'
#df.to_pickle(file_path)

# Creating preprocessed data files

In [ ]:
file_names = os.listdir('./dataset complete/train_landmarks/')
for i in range(0, len(file_names)):
    file_names[i] = file_names[i].replace('.parquet', '')

file_names

In [ ]:
for file in file_names:
    print(file)

    # read in first parquet file
    train_landmarks = pq.read_table('./dataset complete/train_landmarks/' + file + '.parquet')
    train_landmarks = train_landmarks.to_pandas()

    # leave only columns with hand and arm data
    train_landmarks = remove_unwanted_columns(train_landmarks)
    # merge the hand and arm data with the corresponding participant id and phrase
    df = merge_datasets(train_landmarks, df_train)
    # for short nan gaps interpolate the values, else fill Nans with a placeholder number
    df = interpolate_dataset(df)
    # check if all nans have been removed
    if len(df[df.isna().any(axis=1)]) != 0:
        print("Still NANs!")

    file_path = './dataset complete/preprocessed_files/' + file + '.pkl'
    df.to_pickle(file_path)

In [ ]:
file_names = os.listdir('./dataset complete/supplemental_landmarks/')
for i in range(0, len(file_names)):
    file_names[i] = file_names[i].replace('.parquet', '')

file_names

In [ ]:
for file in file_names:
    print(file)

    # read in first parquet file
    train_landmarks = pq.read_table('./dataset complete/supplemental_landmarks/' + file + '.parquet')
    train_landmarks = train_landmarks.to_pandas()

    # leave only columns with hand and arm data
    train_landmarks = remove_unwanted_columns(train_landmarks)
    # merge the hand and arm data with the corresponding participant id and phrase
    df = merge_datasets(train_landmarks, supp_train)
    # for short nan gaps interpolate the values, else fill Nans with a placeholder number
    df = interpolate_dataset(df)
    # check if all nans have been removed
    if len(df[df.isna().any(axis=1)]) != 0:
        print("Still NANs!")

    file_path = './dataset complete/supp_preprocessed_files/' + file + '.pkl'
    df.to_pickle(file_path)

Read in pickle files:

In [ ]:
#df_pkl = pd.read_pickle('./dataset complete/preprocessed_files/933868835.pkl')
#print(len(df_pkl))
#df_pkl

In [ ]:
# Save as csv to have a look if dataset is preprocessed like we wanted
#df_pkl.to_csv('./dataset complete/preprocessed_files/test.csv')
#import os
#path = './dataset complete/preprocessed_files/test.csv'
#os.remove(path)

Merge pickle files

In [ ]:
#df1 = pd.read_pickle('./dataset complete/preprocessed_files/5414471.pkl')
#df2 = pd.read_pickle('./dataset complete/preprocessed_files/105143404.pkl')
#df = pd.concat([df1, df2], axis=0)

# Transformer

In [ ]:
import json

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)
len(characters)

In [ ]:
from transformer import get_compiled_transformer

# TODO: tune hyperparameters
d_model = 128  # hidden layer(s) dimensionality
num_layers = 5 # how many encoders and decoders to stack
num_heads = 8 # how many attention heads should every mha have
ff_dim = 256 # how many neurons shall feed-forward layers have
dropout_rate = 0.1

output_vocab_size = len(characters) + 2

transformer = get_compiled_transformer(
    d_model=d_model,
    num_layers=num_layers,
    num_heads=num_heads,
    ff_dim=ff_dim,
    dropout_rate=dropout_rate,
    output_vocab_size=output_vocab_size
)

In [ ]:
import pandas as pd
df_data = pd.read_pickle('./dataset complete/preprocessed_files/532011803.pkl')
df_data

In [ ]:
# TODO: Tune - the longer the worse is the performance
MAX_PHRASE_LENGTH = 100
MAX_SIGN_LENGTH = 800

In [ ]:
from copy import deepcopy
import numpy as np

phrases = df_data.sequence_id.unique()
columns = ['sequence_id', 'frame', 'participant_id', 'phrase']
targets = []
inputs = []
contexts = []
for i in phrases:
    phrase_table = df_data[df_data['sequence_id'] == i]
    phrase = phrase_table.iloc[0].phrase
    if len(phrase) > MAX_PHRASE_LENGTH:
        print('Houston we have a problem!')
    while len(phrase) < MAX_PHRASE_LENGTH:
        phrase.append(59)
    context = deepcopy(phrase)
    context.insert(0, 60)
    contexts.append(context[:-1])
    targets.append(phrase)

    phrase_table = phrase_table.drop(columns, axis=1)
    phrase_table = phrase_table.to_numpy(copy=True)

    if phrase_table.shape[0] > MAX_SIGN_LENGTH:
        print('Houuuuuussssteeeeeen!')

    phrase_table = np.pad(phrase_table, [(0, MAX_SIGN_LENGTH-phrase_table.shape[0]), (0,0)], 'constant', constant_values=(-1, -1))

    inputs.append(phrase_table)

inputs = np.stack(inputs)
contexts = np.array(contexts)
targets = np.array(targets)
print(inputs.shape)
print(targets.shape)
print(contexts.shape)

In [ ]:
EPOCHS = 5
BATCH_SIZE=64

inputs = [inputs, contexts]

transformer.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)

In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [1]:
import sys
print(sys.version)

3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]


# Train Test Split

In [1]:
import pandas as pd

df_test = pd.read_csv('./dataset complete/train.csv')
df_test

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road
...,...,...,...,...,...
67203,train_landmarks/2118949241.parquet,2118949241,388192924,88,431-366-2913
67204,train_landmarks/2118949241.parquet,2118949241,388225542,154,994-392-3850
67205,train_landmarks/2118949241.parquet,2118949241,388232076,95,https://www.tianjiagenomes.com
67206,train_landmarks/2118949241.parquet,2118949241,388235284,36,90 kerwood circle


In [2]:
p_ids = df_test.participant_id.unique().tolist()
len(p_ids)

94

In [3]:
import random
TRAIN_SPLIT = 0.8
TEST_SPLIT = 0.2

num_train_samples = int(TRAIN_SPLIT * len(p_ids))

train_participants = random.sample(p_ids, num_train_samples)
test_participants = [x for x in p_ids if x not in train_participants]

In [4]:
from copy import deepcopy
import numpy as np
from tqdm import tqdm
import os

# TODO: Tune - the longer the worse is the performance
MAX_PHRASE_LENGTH = 100
MAX_SIGN_LENGTH = 900

columns = ['sequence_id', 'frame', 'participant_id', 'phrase']
targets_train = []
inputs_train = []
contexts_train = []
targets_test = []
inputs_test = []
contexts_test = []

files = os.listdir('./dataset complete/preprocessed_files')

for file in tqdm(files):
    df_data = pd.read_pickle('./dataset complete/preprocessed_files/' + file)
    phrases = df_data.sequence_id.unique()

    for i in phrases:
        phrase_table = df_data[df_data['sequence_id'] == i]

        is_test = False
        if phrase_table.participant_id.unique()[0] in test_participants:
            is_test = True

        phrase = phrase_table.iloc[0].phrase
        if len(phrase) > MAX_PHRASE_LENGTH:
            print('Houston we have a problem!')
        while len(phrase) < MAX_PHRASE_LENGTH:
            phrase.append(59)
        context = deepcopy(phrase)
        context.insert(0, 60)

        phrase_table = phrase_table.drop(columns, axis=1)
        phrase_table = phrase_table.to_numpy(copy=True)

        if phrase_table.shape[0] > MAX_SIGN_LENGTH:
            print('Houuuuuussssteeeeeen!')

        phrase_table = np.pad(phrase_table, [(0, MAX_SIGN_LENGTH-phrase_table.shape[0]), (0,0)], 'constant', constant_values=(-1, -1))

        if not is_test:
            inputs_train.append(phrase_table)
            targets_train.append(phrase)
            contexts_train.append(context[:-1])
        else:
            inputs_test.append(phrase_table)
            targets_test.append(phrase)
            contexts_test.append(context[:-1])

inputs_train = np.stack(inputs_train)
contexts_train = np.array(contexts_train)
targets_train = np.array(targets_train)
inputs_test = np.stack(inputs_test)
contexts_test = np.array(contexts_test)
targets_test = np.array(targets_test)

100%|██████████| 68/68 [01:09<00:00,  1.02s/it]


In [5]:
print(inputs_train.shape)
print(targets_train.shape)
print(contexts_train.shape)

(43110, 900, 144)
(43110, 100)
(43110, 100)


In [6]:
print(inputs_test.shape)
print(targets_test.shape)
print(contexts_test.shape)

(10143, 900, 144)
(10143, 100)
(10143, 100)


In [7]:
import pickle

with open('./dataset complete/train_test_data/train_data.pkl', 'wb') as file:
    train_data = [(inputs_train, contexts_train), targets_train]
    pickle.dump(train_data, file)

with open('./dataset complete/train_test_data/test_data.pkl', 'wb') as file:
    test_data = [(inputs_test, contexts_test), targets_test]
    pickle.dump(test_data, file)